# プロンプトテンプレート

- 著者: [Hye-yoon Jeong](https://github.com/Hye-yoonJeong)
- ピアレビュー : [hyeyeoon](https://github.com/hyeyeoon), [Wooseok Jeong](https://github.com/jeong-wooseok)
- 校閲 : [Q0211](https://github.com/Q0211)
- これは [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial) の一部です

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/02-Prompt/01-PromptTe[...]
[![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/mai[...]

## 概要
このチュートリアルでは、```LangChain``` を用いたプロンプトテンプレートの作成と利用方法について説明します。

プロンプトテンプレートは、会話履歴、構造化された出力、特殊なクエリなど、さまざまなユースケースに対応する動的で柔軟なプロンプトを生成するために不可欠です。

このチュートリアルでは、```PromptTemplate``` オブジェクトの作成、partial 変数の適用、YAML ファイルによるテンプレート管理、```ChatP[...]``` のような高度なツールの活用方法を探ります。

### 目次
- [概要](#overview)
- [環境セットアップ](#environment-setup)
- [PromptTemplate オブジェクトの作成](#creating-a-prompttemplate-object)
- [partial_variables の使用](#using-partial_variables)
- [YAML ファイルからのプロンプトテンプレートの読み込み](#load-prompt-templates-from-yaml-files)
- [ChatPromptTemplate](#chatprompttemplate)
- [MessagePlaceholder](#messageplaceholder)

### 参考資料
- [LangChain ドキュメント : Prompts](https://python.langchain.com/api_reference/core/prompts.html#)
----

## 環境セットアップ

環境をセットアップします。詳細は [Environment Setup](https://wikidocs.net/257836) を参照してください。

**[注意]**
- ```langchain-opentutorial``` はチュートリアル用の便利な環境セットアップ、ユーティリティ関数群を提供するパッケージです。
- 詳細は [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) をご確認ください。

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial


[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 必要なパッケージをインストール
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_community",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [4]:
# 環境変数を設定
from langchain_opentutorial import set_env

set_env(
    {
        # "OPENAI_API_KEY": "",
        # "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "Prompt-Template",
    }
)

環境変数が正常に設定されました。


```ChatOpenAI``` を ```gpt-4o``` モデルで設定します。

In [5]:
from langchain_openai import ChatOpenAI

# モデルを読み込み
llm = ChatOpenAI(model_name="gpt-4o")

## ```PromptTemplate``` オブジェクトの作成

```PromptTemplate``` オブジェクトを作成する方法は2つあります。
- 1. ```from_template()``` メソッドを使用する
- 2. `PromptTemplate` オブジェクトを作成し、プロンプトを同時に指定する

### 方法 1. ```from_template()``` メソッドを使用

- 変数は ```{variable}``` の形式でテンプレート内に定義します。

In [6]:
from langchain_core.prompts import PromptTemplate

# テンプレートを定義します。ここでは {country} が変数です
template = "{country} の首都は何ですか？"

# `from_template` メソッドを使って `PromptTemplate` オブジェクトを作成
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country} の首都は何ですか？')

変数 `country` に値を割り当ててプロンプトを完成させることができます。

In [7]:
# `format` メソッドで変数に値を割り当ててプロンプトを作成
prompt = prompt.format(country="United States of America")
prompt

'United States of America の首都は何ですか？'

In [8]:
# テンプレートを定義
template = "{country} の首都は何ですか？"

# `from_template` メソッドで `PromptTemplate` オブジェクトを作成
prompt = PromptTemplate.from_template(template)

# チェーンを作成
chain = prompt | llm

In [9]:
# country 変数を好きな値に置き換える
chain.invoke("United States of America").content

'アメリカ合衆国の首都はワシントンD.C.です。'

### 方法 2. `PromptTemplate` オブジェクトを作成し、プロンプトを同時に指定

追加の検証のために `input_variables` を明示的に指定します。

そうしないと、テンプレート文字列内の変数と不一致がある場合、インスタンス化時に例外が発生する可能性があります。

In [10]:
# テンプレートを定義
template = "{country} の首都は何ですか？"

# `PromptTemplate` オブジェクトでプロンプトテンプレートを作成
prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country} の首都は何ですか？')

In [11]:
# プロンプトを作成
prompt.format(country="United States of America")

'United States of America の首都は何ですか？'

In [12]:
# テンプレートを定義
template = "{country1} と {country2} の首都はそれぞれどこですか？"

# `PromptTemplate` オブジェクトでプロンプトテンプレートを作成
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "United States of America"  # `partial_variables` は辞書形式で渡す
    },
)
prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': 'United States of America'}, template='{country1} と {country2} の首都はそれぞれどこですか？')

In [13]:
prompt.format(country1="South Korea")

'South Korea と United States of America の首都はそれぞれどこですか？'

In [14]:
prompt_partial = prompt.partial(country2="India")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': 'India'}, template='{country1} と {country2} の首都はそれぞれどこですか？')

In [15]:
prompt_partial.format(country1="South Korea")

'South Korea と India の首都はそれぞれどこですか？'

In [16]:
chain = prompt_partial | llm

In [17]:
chain.invoke("United States of America").content

'アメリカ合衆国の首都はワシントンD.C.、インドの首都はニューデリーです。'

In [18]:
chain.invoke({"country1": "United States of America", "country2": "India"}).content

'アメリカ合衆国の首都はワシントンD.C.、インドの首都はニューデリーです。'

## partial_variables の使用

```partial_variables``` を使うことで部分適用が可能になります。これは**共通の変数**を共有したい場合に特に便利です。

よくある例としては **日付や時刻** です。

例えば、現在の日付をプロンプトに指定したい場合、日付をテンプレートにハードコーディングしたり、他の入力変数と一緒に渡すのは実用的でないことがあります。このような場合、関数を partial_variables に渡すことで動的に日付を挿入できます。

In [19]:
from datetime import datetime

# 現在の日付を出力
datetime.now().strftime("%B %d")

'January 14'

In [20]:
# 今日の日付を返す関数を定義
def get_today():
    return datetime.now().strftime("%B %d")

In [21]:
prompt = PromptTemplate(
    template="今日の日付は {today} です。{n} 人の有名人の誕生日（今日）を挙げてください。生年月日も明記してください。",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # `partial_variables` は辞書形式で渡す
    },
)

In [22]:
# プロンプトを作成
prompt.format(n=3)

"今日の日付は January 14 です。3 人の有名人の誕生日（今日）を挙げてください。生年月日も明記してください。"

In [23]:
# チェーンを作成
chain = prompt | llm

In [24]:
# チェーンを呼び出して結果を確認
print(chain.invoke(3).content)

ここでは、1 月 14 日に生まれた 3 人の有名人を紹介します:

1. **Dave Grohl** - 1969 年 1 月 14 日��まれ。
2. **LL Cool J** - 1968 年 1 月 14 日生まれ。
3. **Jason Bateman** - 1969 年 1 月 14 日生まれ。


In [25]:
# チェーンを呼び出して結果を確認（today を明示的に渡す）
print(chain.invoke({"today": "Jan 02", "n": 3}).content)

ここでは、1 月 2 日に生まれた 3 人の有名人を紹介します:

1. **Cuba Gooding Jr.** - 1968 年 1 月 2 日生まれ。
2. **Taye Diggs** - 1971 年 1 月 2 日生まれ。
3. **Kate Bosworth** - 1983 年 1 月 2 日生まれ。


## YAML ファイルからプロンプトテンプレートを読み込む

プロンプトテンプレートを別々の YAML ファイルで管理し、```load_prompt``` を使って読み込むことができます。

In [26]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml", encoding="utf-8")
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit} の色は何ですか？')

In [27]:
prompt.format(fruit="an apple")

'an apple の色は何ですか？'

In [28]:
prompt2 = load_prompt("prompts/capital.yaml")
print(prompt2.format(country="United States of America"))

アメリカ合衆国の首都についての情報を提供してください。
以下の形式で、300 ワード以内で首都の特徴をまとめてください。
----
[フォーマット]
1. 面積
2. 人口
3. 歴史的名所
4. 地域特産品

#回答:



## ```ChatPromptTemplate```

```ChatPromptTemplate``` は会話履歴をプロンプトに含めるために使���できます。

メッセージは (```role```, ```message```) のタプル形式で構造化され、リストとして作成されます。

**role の種類**
- ```system``` : システム設定用のメッセージ。グローバルな設定関連のプロンプトに使用します。
- ```human``` : ユーザー入力メッセージ。
- ```ai``` : AI の応答メッセージ。

In [29]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country} の首都は何ですか？")
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, pa[...])

In [30]:
chat_prompt.format(country="United States of America")

'Human: United States of America の首都は何ですか？'

In [31]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "You are a friendly AI assistant. Your name is {name}."),
        ("human", "Nice to meet you!"),
        ("ai", "Hello! How can I assist you?"),
        ("human", "{user_input}"),
    ]
)

# チャットメッセージを作成
messages = chat_template.format_messages(name="Teddy", user_input="What is your name?")
messages

[SystemMessage(content='You are a friendly AI assistant. Your name is Teddy.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Nice to meet you!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I assist you?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]

上で作成したメッセージを直接 LLM に渡して呼び出すことができます。

In [32]:
llm.invoke(messages).content

'私の名前は Teddy です。今日はどうお手伝いできますか？'

チェーンを作成して実行することもできます。

In [33]:
chain = chat_template | llm

In [34]:
chain.invoke({"name": "Teddy", "user_input": "What is your name?"}).content

'私の名前は Teddy です。今日はどうお手伝いできますか？'

## MessagePlaceholder

```LangChain``` は `MessagePlaceholder` も提供しており、フォーマット時にメッセージのレンダリングを完全に制御できます。

これは、プロンプトテンプレートでどの役割（role）を使うべきかわからない場合や、フォーマット時にメッセージのリストを挿入したい場合に便利です。

In [35]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a summarization specialist AI assistant. Your mission is to summarize conversations using key points.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "Summarize the conversation so far in {word_count} words."),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(ta[...]]

```MessagesPlaceholder``` を使って会話のメッセージリストを追加できます。

In [36]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "Hello! I’m Teddy. Nice to meet you."),
        ("ai", "Nice to meet you! I look forward to working with you."),
    ],
)

print(formatted_chat_prompt)

System: You are a summarization specialist AI assistant. Your mission is to summarize conversations using key points.
Human: Hello! I’m Teddy. Nice to meet you.
AI: Nice to meet you! I look forward to working with you.
Human: Summarize the conversation so far in 5 words.


In [37]:
# チェーンを作成
chain = chat_prompt | llm | StrOutputParser()

In [ ]:
# チェーンを呼び出して結果を確認
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "Hello! I'm Teddy. Nice to meet you.",
            ),
            ("ai", "Nice to meet you! I look forward to working with you."),
        ],
    }
)

'Teddy は自己紹介をし、挨拶を交わしました。'